In [1]:
# 데이터 처리 및 분석을 위한 라이브러리
import numpy as np  # 수학 연산 및 배열 연산을 위한 라이브러리
import pandas as pd  # 데이터 프레임을 다루기 위한 라이브러리

# 데이터 시각화를 위한 라이브러리
import matplotlib.pyplot as plt  # 그래프 및 차트 그리기
import seaborn as sns  # 시각화 기능을 향상시키는 라이브러리

# 머신러닝 관련 라이브러리
from sklearn.cluster import KMeans  # K-means 클러스터링 알고리즘 (비지도 학습)
from sklearn.preprocessing import LabelEncoder, StandardScaler  # 데이터 전처리를 위한 도구
from sklearn.feature_extraction.text import TfidfVectorizer  # TF-IDF 벡터 변환 (텍스트 데이터 벡터화)
from sklearn.metrics.pairwise import sigmoid_kernel  # 시그모이드 커널을 이용한 유사도 측정
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도를 계산하는 함수

# 추천 시스템 관련 라이브러리 (Surprise 라이브러리 사용)
from surprise import SVD  # SVD(특이값 분해) 기반 추천 시스템 알고리즘
from surprise import Dataset, Reader  # 데이터셋 로딩 및 처리
from surprise.model_selection import train_test_split  # 추천 시스템용 데이터 분할
from surprise import accuracy  # 추천 시스템 평가 (RMSE 등 측정)

# 경고 메시지 무시 (불필요한 경고를 숨기기 위해 사용)
import warnings
warnings.filterwarnings('ignore')

# 자연어 처리 관련 라이브러리
import nltk  # 자연어 처리(NLP)를 위한 라이브러리
import re  # 정규 표현식 (문자열 처리)
import string  # 문자열 관련 기능 제공
from nltk.tokenize import word_tokenize  # 문장을 단어 단위로 토큰화
from nltk.corpus import stopwords  # 불용어(의미 없는 단어) 제거
from nltk.stem import PorterStemmer  # 어간 추출 (동사의 변형을 정규화)

# 실행 시간 측정 (성능 비교 등 활용)
import time

# 최근접 이웃 알고리즘을 위한 라이브러리
from scipy.sparse import csr_matrix  # 희소 행렬(대부분이 0인 행렬) 변환
from sklearn.neighbors import NearestNeighbors  # 최근접 이웃 알고리즘 (KNN 등)


In [2]:
anime = pd.read_csv('./data/anime.csv')

In [3]:
anime.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [4]:
anime.dropna(axis=0, inplace=True)
anime.isna().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [5]:
anime.episodes.value_counts()

episodes
1      5571
2      1075
12      810
13      571
26      514
       ... 
358       1
366       1
201       1
172       1
125       1
Name: count, Length: 187, dtype: int64

In [6]:
duplicated_anime = anime[anime.duplicated()].shape[0] #.shape[0] → 데이터프레임의 행(row) 개수를 의미
#anime[anime.duplicated()] -> duplicated()가 True인 행만 선택해서 새로운 데이터프레임을 만듦.
#anime.duplicated() -> 중복된 행인지(True/False) 확인
print(f'count of duplicate anime: {duplicated_anime}')

count of duplicate anime: 0


In [8]:
def clean_text(text):
    """
        data preprocessing 
    """
    
    # to lowercase
    text = text.lower()

    # remove sybmols and other words
    text = re.sub(r'<[^>]*>', '', text) # <html> 같은 태그 제거
    text = re.sub(r'http\S+', '', text) # URL 제거
    text = re.sub(r'&quot;', '', text) # 특수 기호 제거
    text = re.sub(r'.hack//', '', text) # ".hack//"같은 패턴 제거
    text = re.sub(r'&#039;', '', text) # '&#039;' -> '' (어포스트로피 깨짐 현상 제거)
    text = re.sub(r'A&#039;s', '', text) # A&#039;s -> ''
    text = re.sub(r'I&#039;', 'I\'', text) # 'I&#039;' → 'I\'' (아포스트로피 복구)
    text = re.sub(r'&amp;', 'and', text) # '&amp;' → 'and' (HTML 인코딩 복구)
  
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 4. 숫자 제거 (현재는 주석 처리됨, 필요시 활성화)
    # text = re.sub(r'\d+', '', text)

    # 5. 토큰화 (단어 단위로 분리)
    # words = word_tokenize(text)

    # 6. 불용어 제거 (stopwords)
    # stop_words = set(stopwords.words('english'))
    # words = [word for word in words if word not in stop_words]

    # 7. 어간 추출 (stemming)
    # stemmer = PorterStemmer()
    # words = [stemmer.stem(word) for word in words]

    # 8. 다시 하나의 문자열로 합치기
    # text = ' '.join(words)
    
    return text

In [9]:
start_time = time.time() #time.time()을 사용하여 코드 실행이 시작되는 시간을 저장
anime['name']=anime['name'].apply(clean_text) #df의 "name" 컬럼의 모든 값에 clean_text() 적용
anime['name'] = anime['name'].apply(clean_text) #anime 데이터프레임에도 동일한 작업 수행
end_time = time.time() #실행이 끝나는 시점의 시간 기록
elapsed_time = end_time - start_time #실행 시간을 초 단위로 계산 
print("process time: ", elapsed_time, " sec.")

process time:  0.1530613899230957  sec.


In [10]:
anime.to_csv("./data/anime_v2.csv", index=False)